# Cleaning and Analysis of Craigslist Used Car Postings
Data retrieved from [Kaggle](https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data), due to the dataset being to large to upload to github, manual download is required. Place 'vehicles.csv' in data folder.

Uses : ![Jupyter Notebook](https://img.shields.io/badge/jupyter-%23FA0F00.svg?style=for-the-badge&logo=jupyter&logoColor=white)![Python](https://img.shields.io/badge/python-3670A0?style=for-the-badge&logo=python&logoColor=ffdd54)![Matplotlib](https://img.shields.io/badge/Matplotlib-%23ffffff.svg?style=for-the-badge&logo=Matplotlib&logoColor=black)![NumPy](https://img.shields.io/badge/numpy-%23013243.svg?style=for-the-badge&logo=numpy&logoColor=white)![Pandas](https://img.shields.io/badge/pandas-%23150458.svg?style=for-the-badge&logo=pandas&logoColor=white)

In [1]:
import pandas as pd
import os

In [2]:
data = pd.read_csv("data/vehicles.csv",header=0,index_col=0)
data


,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
id,,,,,,,,,,,,,,,,,,,,,
7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,...,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,...,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,...,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600


To start with we'll look at how much of the data is invalid from the start.

In [3]:
nullCount = pd.DataFrame(columns=["NaNCount","NaN%"],index=data.columns)
for col in data.columns:
	count = len(data) - len(data[col].dropna())
	percent = (count / len(data)) * 100.0
	nullCount.loc[col] = [count,percent]

nullCount.sort_values(by="NaNCount",axis="index", inplace=True)
nullCount


,NaNCount,NaN%
url,0,0.0
region,0,0.0
region_url,0,0.0
price,0,0.0
state,0,0.0
image_url,68,0.01593
posting_date,68,0.01593
description,70,0.016398
year,1205,0.282281
transmission,2556,0.598763


Given that the 'county' column is always null, we can safely delete it.

In [4]:
_ = data.drop("county", inplace=True, axis = 1)

Next, we'll look at trimming outliers

In [5]:
def _forEachPrice(x,L,R):
	if x < L or x > R:
		return None
	else:
		return x

def removeOutliers(series: pd.Series,inplace = False):
	if inplace == False:
		series = series.copy()
	Q1 = series.quantile(0.25)
	Q3 = series.quantile(0.75)
	IQR = Q3 - Q1
	L = Q1 - 1.5 * IQR
	R = Q3 + 1.5 * IQR
	series = series.apply(_forEachPrice, args=(L,R,))
	return series

In [6]:
priceWithoutOutliers = removeOutliers(data['price'])
print("Nulled " , len(data['price'].dropna()) - len(priceWithoutOutliers.dropna()), " values.")

Nulled  8177  values.


Posting dates are stored at strings with the format 'YYYY-MM-DDTHH:MM:SS-????', we'll want them in a datetime format for future use.

In [7]:
#Due to long conversion times, caching is nesesary
cachePath = "data/postingDates.cache"
if os.path.exists(cachePath):
	postingDates = pd.read_pickle(cachePath)
else:
	# WARNING this cell takes a long time to run, my example takes 9 minutes
	postingDates = pd.to_datetime(data["posting_date"],infer_datetime_format=True)
	postingDates.to_pickle(cachePath)

In [8]:
postingDates

id
7222695916                          NaN
7218891961                          NaN
7221797935                          NaN
7222270760                          NaN
7210384030                          NaN
                        ...            
7301591192    2021-04-04 03:21:31-06:00
7301591187    2021-04-04 03:21:29-06:00
7301591147    2021-04-04 03:21:17-06:00
7301591140    2021-04-04 03:21:11-06:00
7301591129    2021-04-04 03:21:07-06:00
Name: posting_date, Length: 426880, dtype: object